In [14]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark = SparkSession.builder.master('local').config('spark.driver.memory', '8g').appName('Ready_for_analysis').getOrCreate()



In [15]:
df = spark.read.parquet(
    '/Users/gabriele.sabato/PycharmProjects/raw_data/price_elasticity_model_data/part-*',
    header=True)


In [16]:
df.createOrReplaceTempView('model_data_table')


In [4]:
df.show()

+------------------+-----------+--------------+-----------+-----+
|         item_code|update_date|delivery_weeks| item_price|sales|
+------------------+-----------+--------------+-----------+-----+
|000000001000000008| 2019-01-26|             1|      13.99|    0|
|000000001000000008| 2019-04-12|             0|      13.99|    0|
|000000001000000008| 2019-06-22|             1|      13.99|    0|
|000000001000000008| 2019-08-13|             1|      13.99|    0|
|000000001000000008| 2019-12-27|             0|      13.99|    0|
|000000001000000013| 2018-12-18|             1|      39.99|    0|
|000000001000000013| 2020-01-30|             1|      39.99|    0|
|000000001000000013| 2020-10-26|             0|31.98999999|    0|
|000000001000000013| 2020-10-27|             0|31.98999999|    0|
|000000001000000014| 2019-04-19|             1|74.98999999|    0|
|000000001000000014| 2019-06-16|             1|74.98999999|    0|
|000000001000000014| 2019-11-10|             0|74.98999999|    0|
|000000001

In [17]:
df_d_item = spark.read.parquet('/Users/gabriele.sabato/PycharmProjects/raw_data/redshift_tables/d_item/*')


In [18]:
df_d_item.createOrReplaceTempView('d_item_tmp')

In [19]:
df_item_visits = spark.read.parquet(
    '/Users/gabriele.sabato/PycharmProjects/raw_data/redshift_tables/item_visits/*2020*/*.parquet',header=True)

In [28]:
df_f_orders = spark.read.parquet(
    '/Users/gabriele.sabato/PycharmProjects/raw_data/redshift_tables/f_orders/*.parquet',header=True)

df_f_orders.createOrReplaceTempView('f_orders')

In [29]:
df_d_calendar = spark.read.parquet(
    '/Users/gabriele.sabato/PycharmProjects/raw_data/redshift_tables/d_calendar/*.parquet',header=True)

df_d_calendar.createOrReplaceTempView('d_calendar')



In [30]:
df_d_order_flags = spark.read.parquet(
    '/Users/gabriele.sabato/PycharmProjects/raw_data/redshift_tables/d_order_flags/*.parquet',header=True)

df_d_order_flags.createOrReplaceTempView('d_order_flags')



In [31]:
df_list_top_seller = spark.sql("""
SELECT di.item_code,
       sum(fo.amount + amount_discount) AS order_amount

  FROM f_orders as fo
       INNER JOIN d_calendar c ON c.date_skey = fo.order_date_skey
       INNER JOIN d_item_tmp di ON di.item_skey = fo.item_skey
INNER JOIN d_order_flags dof on dof.order_flags_skey = fo.order_flags_skey

 WHERE c.date >= current_date - 1*interval ' 1 year'
and dof.is_cancelled_immediately = 'is not cancelled immediately'
and dof.is_cancelled_after_payment = 'is not cancelled after payment'
and dof.is_cancelled_before_payment = 'is not cancelled before payment'
group by di.item_code
ORDER BY order_amount DESC""")

In [37]:
df_list_top_seller.createOrReplaceTempView('top_seller_list')

df_list_top_seller.show()



+------------------+--------------------+
|         item_code|        order_amount|
+------------------+--------------------+
|000000001000021539|883695.9542000000...|
|000000001000054983|754818.1985000000...|
|000000001000016021|734503.7719999999...|
|000000001000021540|724137.2758999999...|
|000000001000054984|720963.9100000000...|
|000000001000062031|584247.1772999999...|
|000000001000016133|565349.0766999999...|
|000000001000015496|534943.0500000000...|
|000000001000113432|512013.6549000000...|
|000000001000073170|497056.6000000000...|
|000000001000021718|479586.3300000000...|
|000000001000015484|478394.0504000000...|
|000000001000015488|467617.6006000000...|
|000000001000021719|456252.3015000000...|
|000000000000001213|452933.6879000000...|
|000000001000008754|449844.4400000000...|
|000000001000008809|449195.9225000000...|
|000000001000055017|441695.1853000000...|
|000000001000005892|438083.9700000000...|
|000000001000121274|434774.4500000000...|
+------------------+--------------

In [38]:
df_list_top_seller_10K= spark.sql("""
SELECT item_code
from top_seller_list
LIMIT 10000""")



In [58]:
top_seller_list_string_10k = df_list_top_seller_10K.rdd.map(lambda x: x.item_code).collect()


In [72]:
#DEFINE SIZE
Number_of_items = 10
selected_items = top_seller_list_string_10k[:Number_of_items]

In [73]:
selected_items[0]= """'""" + selected_items[0]
selected_items[-1]= selected_items[-1] + """'"""

In [74]:
separator =  """' , '"""
final_string = separator.join(selected_items)
print(final_string)


'000000001000021539' , '000000001000054983' , '000000001000016021' , '000000001000021540' , '000000001000054984' , '000000001000062031' , '000000001000016133' , '000000001000015496' , '000000001000113432' , '000000001000073170'


In [75]:
#top_seller_list_string = """'000000001000016021','000000001000016133','000000001000022708', '000000001000015958',
#                           '000000001000016020','000000001000015959','000000001000016019','000000001000022573','000000001000015944',
#                            '000000001000024281'"""

In [90]:
df_item_visits.createOrReplaceTempView('item_visits')

df_item_visits_df = spark.sql("""SELECT item_code,
                                         meta_date,
                                        SUM(unique_views) as all_unique_views
                                    from item_visits
                                  where item_code in ({lista})
                                  GROUP BY item_code, meta_date""".format(lista=final_string))
df_item_visits_df.show()

+------------------+----------+----------------+
|         item_code| meta_date|all_unique_views|
+------------------+----------+----------------+
|000000001000021540|2020-10-24|             316|
|000000001000113432|2020-10-20|             433|
|000000001000054983|2020-01-19|             295|
|000000001000054984|2020-05-23|             431|
|000000001000054984|2020-01-28|             822|
|000000001000015496|2020-03-08|             670|
|000000001000113432|2020-03-30|             298|
|000000001000062031|2020-03-07|             334|
|000000001000054983|2020-06-09|             123|
|000000001000113432|2020-07-01|             218|
|000000001000113432|2020-03-20|             183|
|000000001000073170|2020-11-09|             730|
|000000001000113432|2020-08-29|             307|
|000000001000021540|2020-01-08|             491|
|000000001000054984|2020-12-05|             195|
|000000001000016133|2020-09-22|             309|
|000000001000021540|2020-02-10|             184|
|000000001000062031|

In [91]:
df2 = spark.sql("""SELECT * from model_data_table where update_date >= '2020-01-01' and item_code in ({lista}) ORDER BY update_date""".format(lista=final_string))
df2.show()

+------------------+-----------+--------------+------------+-----+
|         item_code|update_date|delivery_weeks|  item_price|sales|
+------------------+-----------+--------------+------------+-----+
|000000001000021540| 2020-01-01|             2|     1449.99|    3|
|000000001000016021| 2020-01-01|             5|119.98999999|   23|
|000000001000062031| 2020-01-01|             2|     1899.99|    0|
|000000001000054983| 2020-01-01|             2|      749.99|    0|
|000000001000054984| 2020-01-01|             2|      749.99|    1|
|000000001000113432| 2020-01-01|             2|      849.99|    1|
|000000001000073170| 2020-01-01|             2|      429.99|    2|
|000000001000021539| 2020-01-01|             2|     1449.99|    3|
|000000001000016133| 2020-01-01|             1|119.98999999|    3|
|000000001000015496| 2020-01-01|             2|      699.99|    3|
|000000001000021539| 2020-01-02|             2|     1449.99|    1|
|000000001000016021| 2020-01-02|             1|119.98999999|  

In [92]:
df4 = spark.sql ("""SELECT item_code, item_main_category, item_sub_category_1, item_sub_category_2,item_parent_item_code from d_item_tmp where item_code in ({lista})""".format(lista=final_string))
df4.show()

+------------------+--------------------+-------------------+-------------------+---------------------+
|         item_code|  item_main_category|item_sub_category_1|item_sub_category_2|item_parent_item_code|
+------------------+--------------------+-------------------+-------------------+---------------------+
|000000001000015496|             POLSTER|        EINZELSOFAS|        EINZELSOFAS|   000000008000001000|
|000000001000016021|ESSZIMMERSTÜHLE&B...|    ESSZIMMERSTÜHLE|          KLASSISCH|   000000008000001083|
|000000001000016133|ESSZIMMERSTÜHLE&B...|    ESSZIMMERSTÜHLE|          KLASSISCH|   000000008000001083|
|000000001000021539|             POLSTER|   WOHNLANDSCHAFTEN|   WOHNLANDSCHAFTEN|   000000008000001826|
|000000001000021540|             POLSTER|   WOHNLANDSCHAFTEN|   WOHNLANDSCHAFTEN|   000000008000001826|
|000000001000054983|             POLSTER|           ECKSOFAS|          LONGCHAIR|   000000008000006211|
|000000001000054984|             POLSTER|           ECKSOFAS|   

In [93]:
df4.createOrReplaceTempView('d_item_filtered')
df2.createOrReplaceTempView('model_data_table_YTD')
df_item_visits_df.createOrReplaceTempView('item_views_YTD')

In [94]:
df_item_visits_df.describe(['item_code']).show()

+-------+------------------+
|summary|         item_code|
+-------+------------------+
|  count|              3420|
|   mean|    1.0000449329E9|
| stddev| 31004.85622422763|
|    min|000000001000015496|
|    max|000000001000113432|
+-------+------------------+



In [95]:
sql_price_visit_join= """ SELECT mdt_YTD.*, it_YTD.all_unique_views FROM model_data_table_YTD as mdt_YTD LEFT JOIN item_views_YTD as it_YTD
on mdt_YTD.item_code = it_YTD.item_code and mdt_YTD.update_date = it_YTD.meta_date """

print(sql_price_visit_join)

 SELECT mdt_YTD.*, it_YTD.all_unique_views FROM model_data_table_YTD as mdt_YTD LEFT JOIN item_views_YTD as it_YTD
on mdt_YTD.item_code = it_YTD.item_code and mdt_YTD.update_date = it_YTD.meta_date 


In [96]:
df_price_visit_join = spark.sql(sql_price_visit_join)
df_price_visit_join.createOrReplaceTempView('mdt_YTD_it')

df_price_visit_join.describe(['item_code']).show()


+-------+------------------+
|summary|         item_code|
+-------+------------------+
|  count|              3440|
|   mean|    1.0000449329E9|
| stddev|31004.829862644332|
|    min|000000001000015496|
|    max|000000001000113432|
+-------+------------------+



In [97]:
sql_top_item_query_group_by = """
WITH tmp_tbl AS (
      SELECT *,
             CASE
                 WHEN lag(delivery_weeks, 1) OVER (PARTITION BY item_code ORDER BY update_date ASC) =
                      delivery_weeks
                     AND
                      lag(item_price, 1) OVER (PARTITION BY item_code ORDER BY update_date ASC) =
                      item_price
                     THEN NULL
                 ELSE RANK() OVER (PARTITION BY item_code ORDER BY update_date)
                 END AS ranking_col --filled with row number or delivery week if the previous one is part of the same group,
        FROM mdt_YTD_it
  ),
       tmp_tbl2 AS (
           SELECT update_date,
                  item_code,
                  sales,
                  all_unique_views,
                  item_price,
                  delivery_weeks,
                  CASE
                      WHEN ranking_col IS NULL
                          THEN last(ranking_col, True) OVER (PARTITION BY item_code ORDER BY update_date ROWS BETWEEN UNBOUNDED PRECEDING and 1 PRECEDING)
                      ELSE ranking_col
                      END AS ranks
             FROM tmp_tbl
       )
SELECT CAST(MIN(update_date) as date)                     AS min_date,
       CAST (MAX(update_date) as date)                    AS max_date,
       item_code,
       item_price,
       delivery_weeks,
       avg(sales) as avg_sales,
       avg(all_unique_views) as avg_unique_views,
       CASE WHEN sum(all_unique_views) <> 0 THEN sum(sales)/sum(all_unique_views) ELSE NULL END as conv_rate,
       CAST (datediff(MAX(update_date), MIN(update_date) ) + 1 as int) AS bin
  FROM tmp_tbl2
 GROUP BY ranks, delivery_weeks, item_price, item_code
 ORDER BY item_code, min_date ASC;
"""

In [98]:
print(sql_top_item_query_group_by)


WITH tmp_tbl AS (
      SELECT *,
             CASE
                 WHEN lag(delivery_weeks, 1) OVER (PARTITION BY item_code ORDER BY update_date ASC) =
                      delivery_weeks
                     AND
                      lag(item_price, 1) OVER (PARTITION BY item_code ORDER BY update_date ASC) =
                      item_price
                     THEN NULL
                 ELSE RANK() OVER (PARTITION BY item_code ORDER BY update_date)
                 END AS ranking_col --filled with row number or delivery week if the previous one is part of the same group,
        FROM mdt_YTD_it
  ),
       tmp_tbl2 AS (
           SELECT update_date,
                  item_code,
                  sales,
                  all_unique_views,
                  item_price,
                  delivery_weeks,
                  CASE
                      WHEN ranking_col IS NULL
                          THEN last(ranking_col, True) OVER (PARTITION BY item_code ORDER BY update_date ROWS B

In [99]:
sql_top_item_df = spark.sql(sql_top_item_query_group_by)

In [100]:
sql_top_item_df.show(5000,False)

+----------+----------+------------------+------------+--------------+------------------+------------------+---------------------+---+
|min_date  |max_date  |item_code         |item_price  |delivery_weeks|avg_sales         |avg_unique_views  |conv_rate            |bin|
+----------+----------+------------------+------------+--------------+------------------+------------------+---------------------+---+
|2020-01-01|2020-01-07|000000001000015496|699.99      |2             |3.7142857142857144|565.2857142857143 |0.006570634318928481 |7  |
|2020-01-08|2020-02-05|000000001000015496|659.99      |2             |1.5172413793103448|450.86206896551727|0.003365200764818356 |29 |
|2020-02-06|2020-03-07|000000001000015496|699.99      |2             |2.2903225806451615|374.6774193548387 |0.006112785191562635 |31 |
|2020-03-08|2020-03-09|000000001000015496|699.99      |3             |3.5               |550.0             |0.006363636363636364 |2  |
|2020-03-10|2020-03-13|000000001000015496|699.99      |

In [101]:
#CREATE A TABLE VIEW FOR Top10 sellers
sql_top_item_df.createOrReplaceTempView('top_sellers')


In [102]:
#JOIN THE TWO TABLEs
sql_join_query = " SELECT top.*, di.item_parent_item_code, di.item_main_category, di.item_sub_category_1, di.item_sub_category_2" \
                 "        from top_sellers as top " \
                 "INNER JOIN d_item_filtered as di on di.item_code = top.item_code "\
                 "ORDER BY top.min_date ASC"

In [103]:
print(sql_join_query)


 SELECT top.*, di.item_parent_item_code, di.item_main_category, di.item_sub_category_1, di.item_sub_category_2        from top_sellers as top INNER JOIN d_item_filtered as di on di.item_code = top.item_code ORDER BY top.min_date ASC


In [104]:
final_top_seller_df = spark.sql(sql_join_query)

In [105]:
final_top_seller_df.show(3000,False)


+----------+----------+------------------+------------+--------------+------------------+------------------+---------------------+---+---------------------+---------------------+-------------------+-------------------+
|min_date  |max_date  |item_code         |item_price  |delivery_weeks|avg_sales         |avg_unique_views  |conv_rate            |bin|item_parent_item_code|item_main_category   |item_sub_category_1|item_sub_category_2|
+----------+----------+------------------+------------+--------------+------------------+------------------+---------------------+---+---------------------+---------------------+-------------------+-------------------+
|2020-01-01|2020-01-07|000000001000054983|749.99      |2             |1.5714285714285714|510.42857142857144|0.0030786453960257487|7  |000000008000006211   |POLSTER              |ECKSOFAS           |LONGCHAIR          |
|2020-01-01|2020-01-07|000000001000054984|749.99      |2             |2.142857142857143 |152.14285714285714|0.01408450704225

In [106]:
final_top_seller_df.describe(['bin']).show()

+-------+------------------+
|summary|               bin|
+-------+------------------+
|  count|               471|
|   mean|7.3036093418259025|
| stddev|17.880072341714214|
|    min|                 1|
|    max|               201|
+-------+------------------+



In [107]:
pd_df_top_sellers = final_top_seller_df.toPandas()


In [108]:
import pickle
import pandas as pd
import numpy as np


In [109]:
pd_df_top_sellers.to_pickle('/Users/gabriele.sabato/PycharmProjects/raw_data/DataFrames/Top10_seller_YTD_group_by_item_visit.pickle')

In [110]:
pd_df_top_sellers['log_price'] = np.log(pd_df_top_sellers['item_price']+0.0001)

In [111]:
pd_df_top_sellers['log_delivery_weeks'] = np.log(pd_df_top_sellers['delivery_weeks']+0.0001)

In [112]:
pd_df_top_sellers['log_sales'] = np.log(pd_df_top_sellers['avg_sales']+0.0001)


In [113]:
pd_df_top_sellers['log_avg_unique_views'] = np.log(pd_df_top_sellers['avg_unique_views']+0.0001)


In [114]:
pd_df_top_sellers['log_conv_rate']= np.log(pd_df_top_sellers['conv_rate']+0.0001)

In [115]:
pd_df_top_sellers.to_pickle('/Users/gabriele.sabato/PycharmProjects/raw_data/DataFrames/Top10_seller_YTD_group_by_item_visit.pickle')